In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
import time
from tqdm import tqdm_notebook as tqdm


from process_visualization import processVisualizer

In [2]:
# Загрузка данных
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

##  Пример использования

In [3]:
prc_vis = processVisualizer(full_train, full_test)

### Зеленые - трейн, красные - тест

In [4]:
prc_vis.visualize_objects(train_idxs=full_test['sat_id'].unique()[:100])